# AutogradTensor Development

We have an issue with backprop in PyTorch. When you call `.backward()` on a torch.Tensor it immediately drops down into C++ code. However, we need to keep track of all operations in Python for use in calculating sensitivity as well as other things. We need to override `.backward()` such that we go through the backpropagation chain in Python instead of the C++ layer.

The idea here is we create a `BackpropTensor` that overrides `.backward()`. Maybe a different name, like `GradientTensor` since it should only be used on tensors where we need gradients. We can discuss naming later I suppose. Anyway, this new tensor should override `.backward()`...

In [1]:
import sys
sys.path.append('/Users/atrask/Laboratory/openmined/PySyft')

# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
mat = sy.VirtualWorker(hook, id="matalicious")

In [3]:
a = torch.tensor([3, 2., 0], requires_grad=True)
b = torch.tensor([1, 2., 3], requires_grad=True)

a = a.send(mat, local_autograd=True)
b = b.send(mat, local_autograd=True)

In [4]:
c = a * b

In [5]:
c.backward(torch.tensor([1., 1., 1.]).send(mat))

(Wrapper)>[PointerTensor | me:16086081552 -> matalicious:10300591275]

In [6]:
a.grad

(Wrapper)>[PointerTensor | me:80830519782 -> matalicious:86953182002]::grad

In [7]:
b.grad

(Wrapper)>[PointerTensor | me:95962704234 -> matalicious:28079150969]::grad

In [8]:
mat._objects

{19119428210: tensor([3., 4., 0.], grad_fn=<MulBackward0>),
 28079150969: tensor([1., 2., 3.], requires_grad=True),
 86953182002: tensor([3., 2., 0.], requires_grad=True)}

In [1]:
# import sys
# sys.path.append('/Users/atrask/Laboratory/openmined/PySyft')

# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
a = AutogradTensor().on(torch.tensor([3, 2., 0]))
b = AutogradTensor().on(torch.tensor([1, 2., 3]))
c = a + b

In [3]:
d = c + c

In [4]:
c.grad_fn(torch.ones(3))

(tensor([1., 1., 1.]), tensor([1., 1., 1.]))

In [7]:
c.backward()

AttributeError: 'Tensor' object has no attribute 'child'

In [13]:
a

(Wrapper)>AutogradTensor>tensor([3., 2., 0.])

In [6]:
# need to fix this, should show the gradient
a.grad

In [7]:
a.child.grad

AutogradTensor>tensor([1., 1., 1.])

In [8]:
b.child.grad

tensor([1., 1., 1.])

In [9]:
d = AutogradTensor().on(torch.rand(3))
e = AutogradTensor().on(torch.rand(3))

In [10]:
f = d.sin() * e.sin()

In [11]:
f.grad_fn

MulBackward

In [12]:
f.grad_fn.next_functions

(SinBackward, SinBackward)

In [13]:
f.backward()

In [15]:
e.child.grad

tensor([0.4016, 0.0379, 0.1268])